In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [12]:
# 获取数据
data = pd.read_csv('./data/书籍评价.csv', encoding='gbk')
data

,Unnamed: 0,内容,评价
0,0,从编程小白的角度看，入门极佳。,好评
1,1,很好的入门书，简洁全面，适合小白。,好评
2,2,讲解全面，许多小细节都有顾及，三个小项目受益匪浅。,好评
3,3,前半部分讲概念深入浅出，要言不烦，很赞,好评
4,4,看了一遍还是不会写，有个概念而已,差评
5,5,中规中矩的教科书，零基础的看了依旧看不懂,差评
6,6,内容太浅显，个人认为不适合有其它语言编程基础的人,差评
7,7,破书一本,差评
8,8,适合完完全全的小白读，有其他语言经验的可以去看别的书,差评
9,9,基础知识写的挺好的！,好评


In [13]:
# 数据基本处理
content = data['内容']
content.head()

0              从编程小白的角度看，入门极佳。
1            很好的入门书，简洁全面，适合小白。
2    讲解全面，许多小细节都有顾及，三个小项目受益匪浅。
3          前半部分讲概念深入浅出，要言不烦，很赞
4             看了一遍还是不会写，有个概念而已
Name: 内容, dtype: object

In [14]:
# 把评价中的好评和差评转换成数字
data.loc[data.loc[:,'评价']=='好评', '评论编号'] = 1
data.loc[data.loc[:,'评价']=='差评', '评论编号'] = 0
data

,Unnamed: 0,内容,评价,评论编号
0,0,从编程小白的角度看，入门极佳。,好评,1.0
1,1,很好的入门书，简洁全面，适合小白。,好评,1.0
2,2,讲解全面，许多小细节都有顾及，三个小项目受益匪浅。,好评,1.0
3,3,前半部分讲概念深入浅出，要言不烦，很赞,好评,1.0
4,4,看了一遍还是不会写，有个概念而已,差评,0.0
5,5,中规中矩的教科书，零基础的看了依旧看不懂,差评,0.0
6,6,内容太浅显，个人认为不适合有其它语言编程基础的人,差评,0.0
7,7,破书一本,差评,0.0
8,8,适合完完全全的小白读，有其他语言经验的可以去看别的书,差评,0.0
9,9,基础知识写的挺好的！,好评,1.0


In [20]:
# 选择停用词
stopwords=[]
with open('./data/stopwords.txt','r',encoding='utf-8') as f:
    lines=f.readlines()
    for tmp in lines:
        line=tmp.strip()
        stopwords.append(line)

!
"
#
$
%
&
'
(
)
*
+
,
-
--
.
..
...
......
...................
./
.一
.数
.日
/
//
0
1
2
3
4
5
6
7
8
9
:
://
::
;
<
=
>
>>
?
@
A
Lex
[
\
]
^
_
`
exp
sub
sup
|
}
~
~~~~
·
×
×××
Δ
Ψ
γ
μ
φ
φ．
В
—
——
———
‘
’
’‘
“
”
”，
…
……
…………………………………………………③
′∈
′｜
℃
Ⅲ
↑
→
∈［
∪φ∈
≈
①
②
②ｃ
③
③］
④
⑤
⑥
⑦
⑧
⑨
⑩
──
■
▲

、
。
〈
〉
《
》
》），
」
『
』
【
】
〔
〕
〕〔
㈧
一
一.
一一
一下
一个
一些
一何
一切
一则
一则通过
一天
一定
一方面
一旦
一时
一来
一样
一次
一片
一番
一直
一致
一般
一起
一转眼
一边
一面
七
万一
三
三天两头
三番两次
三番五次
上
上下
上升
上去
上来
上述
上面
下
下列
下去
下来
下面
不
不一
不下
不久
不了
不亦乐乎
不仅
不仅...而且
不仅仅
不仅仅是
不会
不但
不但...而且
不光
不免
不再
不力
不单
不变
不只
不可
不可开交
不可抗拒
不同
不外
不外乎
不够
不大
不如
不妨
不定
不对
不少
不尽
不尽然
不巧
不已
不常
不得
不得不
不得了
不得已
不必
不怎么
不怕
不惟
不成
不拘
不择手段
不敢
不料
不断
不日
不时
不是
不曾
不止
不止一次
不比
不消
不满
不然
不然的话
不特
不独
不由得
不知不觉
不管
不管怎样
不经意
不胜
不能
不能不
不至于
不若
不要
不论
不起
不足
不过
不迭
不问
不限
与
与其
与其说
与否
与此同时
专门
且
且不说
且说
两者
严格
严重
个
个人
个别
中小
中间
丰富
串行
临
临到
为
为主
为了
为什么
为什麽
为何
为止
为此
为着
主张
主要
举凡
举行
乃
乃至
乃至于
么
之
之一
之前
之后
之後
之所以
之类
乌乎
乎
乒
乘
乘势
乘机
乘胜
乘虚
乘隙
九
也
也好
也就是说
也是
也罢
了
了解
争取
二
二来
二话不说
二话没说
于
于是
于是乎
云云
云尔
互
互相
五
些
交口
亦
产生
亲口
亲手
亲眼
亲

In [24]:
stopwords = list(set(stopwords))
stopwords

['',
 '允许',
 '构成',
 '您是',
 '不独',
 '当时',
 '顷',
 '［②ｇ］',
 '尽量',
 '儿',
 '保管',
 '得到',
 '日臻',
 '看',
 '么',
 '所在',
 '且',
 '０：２',
 '复杂',
 '喀',
 '前者',
 '愿意',
 '】',
 '而又',
 '如',
 '除',
 '来讲',
 '了',
 '假使',
 '...',
 '忽地',
 '使',
 '怎么办',
 '因',
 '独',
 '极大',
 '相应',
 '不拘',
 '进步',
 '十分',
 '比较',
 '立刻',
 '这一来',
 '》',
 '无法',
 '从事',
 '这时',
 '起来',
 '到处',
 '倘',
 '［②③］',
 '且不说',
 '掌握',
 '传',
 '往往',
 '三番五次',
 '有力',
 'φ',
 '傥然',
 '反倒',
 '地',
 '独自',
 '除此而外',
 '［⑤ｅ］',
 '换句话说',
 '相对',
 '第',
 '怎奈',
 '单纯',
 '７',
 '随后',
 '便',
 '咦',
 '.',
 '一样',
 '〈',
 '最后',
 '来不及',
 '吱',
 '存在',
 '［⑧］',
 '常常',
 '5',
 '如若',
 '据说',
 '甚么',
 '不得已',
 '不若',
 '#',
 '不得不',
 '倘然',
 '仅仅',
 '唉',
 '串行',
 '其次',
 '倘若',
 '借以',
 '莫不然',
 '关于',
 '为',
 '突出',
 '够瞧的',
 '＆',
 '坚决',
 '每时每刻',
 '别管',
 '日益',
 '长此下去',
 '朝着',
 '奇',
 '大略',
 '但凡',
 '乌乎',
 '从头',
 '不怎么',
 '你',
 '连声',
 '处在',
 '截至',
 '遵循',
 '相当',
 '充其极',
 '何必',
 '敢于',
 '上下',
 '砰',
 '％',
 '看上去',
 '并',
 '猛然',
 '注意',
 '精光',
 '一次',
 '任凭',
 '之一',
 '［②②］',
 '伟大',
 '］∧′＝［',
 '──',
 '巩固',
 '何处',
 '各式',
 '出于',
 

In [26]:
# 把'内容'处理，转换成标准格式
comment_list = []

for tmp in content:
    print(tmp)
    # 对文本进行切割
    seg_list = jieba.cut(tmp, cut_all=False)
    seg_str = ','.join(seg_list)
    print(seg_str)
    comment_list.append(seg_str)

Building prefix dict from the default dictionary ...


 从编程小白的角度看，入门极佳。


Dumping model to file cache C:\Users\ghdong\AppData\Local\Temp\jieba.cache
Loading model cost 0.531 seconds.
Prefix dict has been built successfully.


 ,从,编程,小白,的,角度看,，,入门,极佳,。
很好的入门书，简洁全面，适合小白。
很,好,的,入门,书,，,简洁,全面,，,适合,小白,。
讲解全面，许多小细节都有顾及，三个小项目受益匪浅。
讲解,全面,，,许多,小,细节,都,有,顾及,，,三个,小,项目,受益匪浅,。
前半部分讲概念深入浅出，要言不烦，很赞
前半部,分讲,概念,深入浅出,，,要言不烦,，,很赞
看了一遍还是不会写，有个概念而已
看,了,一遍,还是,不会,写,，,有个,概念,而已
中规中矩的教科书，零基础的看了依旧看不懂
中规中矩,的,教科书,，,零,基础,的,看,了,依旧,看不懂
内容太浅显，个人认为不适合有其它语言编程基础的人
内容,太,浅显,，,个人,认为,不,适合,有,其它,语言,编程,基础,的,人
破书一本
破书,一本
适合完完全全的小白读，有其他语言经验的可以去看别的书
适合,完完全全,的,小白读,，,有,其他,语言,经验,的,可以,去,看,别的,书
基础知识写的挺好的！
基础知识,写,的,挺,好,的,！
太基础
太,基础
略_嗦。。适合完全没有编程经验的小白
略,_,嗦,。,。,适合,完全,没有,编程,经验,的,小白
真的真的不建议买
真的,真的,不,建议,买


In [28]:
# 统计词个数
con = CountVectorizer(stop_words=stopwords)
x = con.fit_transform(comment_list)
x.toarray()

D:\miniconda\envs\PythonProject\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(


array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0

In [30]:
con.get_feature_names_out()

array(['一本', '一遍', '三个', '中规中矩', '依旧', '入门', '内容', '分讲', '前半部', '受益匪浅',
       '基础', '基础知识', '完完全全', '小白', '小白读', '建议', '很赞', '教科书', '有个', '极佳',
       '概念', '浅显', '深入浅出', '看不懂', '真的', '破书', '简洁', '细节', '经验', '编程',
       '要言不烦', '角度看', '讲解', '语言', '适合', '项目', '顾及'], dtype=object)

In [36]:
# 准备训练集和测试集
x_train = x.toarray()[:10,:]
y_train = data['评价'][:10]

In [37]:
x_test = x.toarray()[10:,:]
y_test = data['评价'][10:]

In [42]:
# 训练模型
mb = MultinomialNB(alpha=1)
mb.fit(x_train, y_train)
y_predict = mb.predict(x_test)

print(y_test)
print(y_predict)

10    差评
11    差评
12    差评
Name: 评价, dtype: object
['差评' '差评' '差评']


In [40]:
# 模型评估
mb.score(x_test, y_test)

1.0